In [1]:
import pickle
import re
import math
from docx2python import docx2python
import pandas as pd
pd.set_option('display.max_columns', None)

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
doc_result = docx2python("Categories.docx")
categories_to_subcategories = {}
k = None
for i in doc_result.body[0][0][0]:
    if len(re.findall('\t', i)) == 0:
        k = ' '.join(re.findall(r'\w+', i))
        categories_to_subcategories[k] = [k]
    
    categories_to_subcategories[k].append(' '.join(re.findall(r'\w+', i)))

subcategories_to_categories = {}
for k, v in categories_to_subcategories.items():
    for x in set(v):
        subcategories_to_categories.setdefault(x.lower(), []).append(k.lower())

# https://stackoverflow.com/questions/54481198/python-match-multiple-substrings-in-a-string
def clean_categories(string):
    if type(string) is str:
        categories = []
        for key, values in categories_to_subcategories.items():
            pattern = r'\b({})\b'.format('|'.join(map(re.escape, values)))
            matches = set(map(str.lower, re.findall(pattern, string, re.IGNORECASE)))
            categories.extend(list(filter(None, [x.lower() for x in values if x.lower() in matches])))
        return list(set(categories))
    else:
        return string

modift nlargest to filter top k

In [8]:
businesses_df = pd.read_csv('../dataset/business.csv')

# Dropping hours and attributes as their sub dictionaries are already added as features.
businesses_df.drop(['Unnamed: 0', 'hours', 'attributes'], axis=1, inplace=True)
businesses_df = businesses_df[businesses_df['is_open'] == 1]

businesses_df['rating_category'] = businesses_df['stars'].apply(lambda x: 'low' if x < 3 else ('medium' if x < 4 else 'high'))

businesses_df = businesses_df[businesses_df['state'].isin(businesses_df.groupby(by='state')['business_id'].count().sort_values(ascending=False).nlargest(3).keys().values)]

businesses_df['cleaned_categories'] = businesses_df['categories'].apply(clean_categories)
businesses_df['cleaned_categories'] = businesses_df['cleaned_categories'].apply(lambda x: x if type(x) is list else math.nan)
businesses_df['parent_categories'] = businesses_df['cleaned_categories'].apply(lambda x: list(set([subcategories_to_categories[k][0] for k in x])) if type(x) is list else x)

selected_parent_categories = businesses_df[['parent_categories', 'business_id']].explode('parent_categories').reset_index()[['parent_categories', 'business_id']].groupby(
    by='parent_categories')['business_id'].count().nlargest(3).index.values

businesses_df = businesses_df[businesses_df['business_id'].isin(
    pd.concat([businesses_df[['parent_categories', 'business_id']].explode('parent_categories').reset_index()[['parent_categories', 'business_id']].groupby(
        by='parent_categories').get_group(name)['business_id'] for name in selected_parent_categories]))]

businesses_df.to_pickle("business_curtailed.pkl")
businesses_df['business_id'].to_pickle("business_ids.pkl")

del(businesses_df)

/geode2/home/u100/vimohan/Carbonate/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,9,26,27,32,38,44,59,80,93,104) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
users_df = pd.read_csv('../dataset/user.csv')
users_df.drop(['Unnamed: 0'], axis=1, inplace=True)
users_df['year'] = pd.DatetimeIndex(pd.to_datetime(users_df['yelping_since'], format='%Y-%m-%d %H:%M:%S')).year
users_df = users_df[users_df['year'] >= 2010]
users_df.to_pickle("user_curtailed.pkl")

users_df['user_id'].to_pickle("user_ids.pkl")
del(users_df)

/geode2/home/u100/vimohan/Carbonate/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
user_ids = pd.read_pickle('../pkl/user_ids.pkl')
business_ids = pd.read_pickle('../pkl/business_ids.pkl')
reviews_df = pd.read_csv('../dataset/review.csv')
# reviews_df.drop(['cool', 'useful', 'funny'], axis=1, inplace=True)
for c in reviews_df.columns:
    reviews_df[c] = reviews_df[c].apply(lambda x: x[2:-1] if type(x) is str else x)
reviews_df['date'] = pd.to_datetime(pd.to_datetime(reviews_df['date'], format='%Y-%m-%d %H:%M:%S').dt.date, format='%Y-%m-%d')
reviews_df = reviews_df[(reviews_df['user_id'].isin(user_ids)) & (reviews_df['business_id'].isin(business_ids))]

reviews_df['rating_category'] = reviews_df['stars'].apply(lambda x: 'low' if x < 3 else ('medium' if x < 4 else 'high'))
reviews_df[['rating_category', 'business_id']].groupby(by='rating_category').count()
res = reviews_df[['user_id', 'business_id', 'date', 'text']].groupby(by=['user_id', 'business_id', 'date'])['text'].count().sort_values(ascending=False)
res = res[res > 1].index.values

res = list(zip(*res))
u, b, d = list(res[0]), list(res[1]), list(res[2])

reviews_df = reviews_df[~((reviews_df['user_id'].isin(u)) & (reviews_df['business_id'].isin(b)) & (reviews_df['date'].isin(d)))]

idx = reviews_df.groupby(['user_id', 'business_id'])['date'].transform(max) == reviews_df['date']
reviews_df = reviews_df[idx]

reviews_df.to_pickle("reviews_curtailed.pkl")
reviews_df['review_id'].to_pickle("reviews_id.pkl")
del(reviews_df)

##### Generating a bigger Data set for review text data.

Including data from **Top 7 states** instead of just 3

In [8]:
businesses_df = pd.read_csv('../dataset/business.csv')

# Dropping hours and attributes as their sub dictionaries are already added as features.
businesses_df.drop(['Unnamed: 0', 'hours', 'attributes'], axis=1, inplace=True)
businesses_df = businesses_df[businesses_df['is_open'] == 1]

businesses_df['rating_category'] = businesses_df['stars'].apply(lambda x: 'low' if x < 3 else ('medium' if x < 4 else 'high'))

businesses_df = businesses_df[businesses_df['state'].isin(businesses_df.groupby(by='state')['business_id'].count().sort_values(ascending=False).nlargest(7).keys().values)]

businesses_df['cleaned_categories'] = businesses_df['categories'].apply(clean_categories)
businesses_df['cleaned_categories'] = businesses_df['cleaned_categories'].apply(lambda x: x if type(x) is list else math.nan)
businesses_df['parent_categories'] = businesses_df['cleaned_categories'].apply(lambda x: list(set([subcategories_to_categories[k][0] for k in x])) if type(x) is list else x)

selected_parent_categories = businesses_df[['parent_categories', 'business_id']].explode('parent_categories').reset_index()[['parent_categories', 'business_id']].groupby(
    by='parent_categories')['business_id'].count().nlargest(3).index.values

businesses_df = businesses_df[businesses_df['business_id'].isin(
    pd.concat([businesses_df[['parent_categories', 'business_id']].explode('parent_categories').reset_index()[['parent_categories', 'business_id']].groupby(
        by='parent_categories').get_group(name)['business_id'] for name in selected_parent_categories]))]

businesses_df.to_pickle("business_curtailed_4gb.pkl")
businesses_df['business_id'].to_pickle("business_ids_4gb.pkl")

del(businesses_df)

/geode2/home/u100/vimohan/Carbonate/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,9,26,27,32,38,44,59,80,93,104) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
users_df = pd.read_csv('../dataset/user.csv')
users_df.drop(['Unnamed: 0'], axis=1, inplace=True)
users_df['year'] = pd.DatetimeIndex(pd.to_datetime(users_df['yelping_since'], format='%Y-%m-%d %H:%M:%S')).year
users_df = users_df[users_df['year'] >= 2010]
users_df.to_pickle("user_curtailed_4gb.pkl")

users_df['user_id'].to_pickle("user_ids_4gb.pkl")
del(users_df)

/geode2/home/u100/vimohan/Carbonate/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
user_ids = pd.read_pickle('../pkl/user_ids_4gb.pkl')
business_ids = pd.read_pickle('../pkl/business_ids_4gb.pkl')
reviews_df = pd.read_csv('../dataset/review.csv')
# reviews_df.drop(['cool', 'useful', 'funny'], axis=1, inplace=True)
for c in reviews_df.columns:
    reviews_df[c] = reviews_df[c].apply(lambda x: x[2:-1] if type(x) is str else x)
reviews_df['date'] = pd.to_datetime(pd.to_datetime(reviews_df['date'], format='%Y-%m-%d %H:%M:%S').dt.date, format='%Y-%m-%d')
reviews_df = reviews_df[(reviews_df['user_id'].isin(user_ids)) & (reviews_df['business_id'].isin(business_ids))]

reviews_df['rating_category'] = reviews_df['stars'].apply(lambda x: 'low' if x < 3 else ('medium' if x < 4 else 'high'))
reviews_df[['rating_category', 'business_id']].groupby(by='rating_category').count()
res = reviews_df[['user_id', 'business_id', 'date', 'text']].groupby(by=['user_id', 'business_id', 'date'])['text'].count().sort_values(ascending=False)
res = res[res > 1].index.values

res = list(zip(*res))
u, b, d = list(res[0]), list(res[1]), list(res[2])

reviews_df = reviews_df[~((reviews_df['user_id'].isin(u)) & (reviews_df['business_id'].isin(b)) & (reviews_df['date'].isin(d)))]

idx = reviews_df.groupby(['user_id', 'business_id'])['date'].transform(max) == reviews_df['date']
reviews_df = reviews_df[idx]

reviews_df.to_pickle("reviews_curtailed_4gb.pkl")
#reviews_df['review_id'].to_pickle("reviews_id.pkl")
del(reviews_df)